In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.compose import make_column_transformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import OneHotEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier

from sklearn.isotonic import IsotonicRegression as IR
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN

from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.under_sampling import NearMiss

from imblearn.combine import SMOTEENN

from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score, \
                            average_precision_score, accuracy_score, \
                            balanced_accuracy_score, roc_auc_score, \
                            matthews_corrcoef

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, \
    ConfusionMatrixDisplay, PrecisionRecallDisplay, roc_curve, auc

import matplotlib
matplotlib.rcParams.update({'font.size': 22})

import warnings
warnings.filterwarnings('ignore')

In [2]:
!which python

/home/tanmoysarkar/Trustworthiness/SEER/.venv/bin/python


In [3]:
%pwd

'/raid/home/tanmoysarkar/Trustworthiness/Diabetes - Paper/Gender Disparity in Diabetes'

# Load data

In [3]:
brfss_2021_mapped_cat = pd.read_csv('BRFSS Datasets - Cleaned/Mapped categories (Step 2)/BRFSS 2021.csv')

brfss_2021_mapped_cat['Race'] = brfss_2021_mapped_cat['Race'].replace({1: 'White', 2: 'Black', 3: 'American Indian',
                                                               4: 'Asian', 5: 'Native Hawaiian', 6: 'Other race',
                                                               7: 'Multiracial', 8: 'Hispanic'})

brfss_2019_mapped_cat = pd.read_csv('BRFSS Datasets - Cleaned/Mapped categories (Step 2)/BRFSS 2019.csv')

brfss_2019_mapped_cat['Race'] = brfss_2019_mapped_cat['Race'].replace({1: 'White', 2: 'Black', 3: 'American Indian',
                                                               4: 'Asian', 5: 'Native Hawaiian', 6: 'Other race',
                                                               7: 'Multiracial', 8: 'Hispanic'})

brfss_2017_mapped_cat = pd.read_csv('BRFSS Datasets - Cleaned/Mapped categories (Step 2)/BRFSS 2017.csv')

brfss_2017_mapped_cat['Race'] = brfss_2017_mapped_cat['Race'].replace({1: 'White', 2: 'Black', 3: 'American Indian',
                                                               4: 'Asian', 5: 'Native Hawaiian', 6: 'Other race',
                                                               7: 'Multiracial', 8: 'Hispanic'})

brfss_2015_mapped_cat = pd.read_csv('BRFSS Datasets - Cleaned/Mapped categories (Step 2)/BRFSS 2015.csv')

brfss_2015_mapped_cat['Race'] = brfss_2015_mapped_cat['Race'].replace({1: 'White', 2: 'Black', 3: 'American Indian',
                                                               4: 'Asian', 5: 'Native Hawaiian', 6: 'Other race',
                                                               7: 'Multiracial', 8: 'Hispanic'})

print(brfss_2021_mapped_cat.shape, brfss_2019_mapped_cat.shape, brfss_2017_mapped_cat.shape, brfss_2015_mapped_cat.shape
)

brfss_2021_one_hot = pd.read_csv('BRFSS Datasets - Cleaned/One hot encoded (Step 3)/BRFSS 2021.csv')
brfss_2019_one_hot = pd.read_csv('BRFSS Datasets - Cleaned/One hot encoded (Step 3)/BRFSS 2019.csv')
brfss_2017_one_hot = pd.read_csv('BRFSS Datasets - Cleaned/One hot encoded (Step 3)/BRFSS 2017.csv')
brfss_2015_one_hot = pd.read_csv('BRFSS Datasets - Cleaned/One hot encoded (Step 3)/BRFSS 2015.csv')

print(brfss_2021_one_hot.shape, brfss_2019_one_hot.shape, brfss_2017_one_hot.shape, brfss_2015_one_hot.shape)

brfss_2021_one_hot['Race'] = brfss_2021_mapped_cat['Race'] # this is added to test performance over race
brfss_2019_one_hot['Race'] = brfss_2019_mapped_cat['Race'] # this is added to test performance over race
brfss_2017_one_hot['Race'] = brfss_2017_mapped_cat['Race'] # this is added to test performance over race
brfss_2015_one_hot['Race'] = brfss_2015_mapped_cat['Race'] # this is added to test performance over race

print(brfss_2021_one_hot.shape, brfss_2019_one_hot.shape, brfss_2017_one_hot.shape, brfss_2015_one_hot.shape)

(200136, 30) (210874, 30) (238130, 30) (220390, 30)
(200136, 49) (210874, 49) (238130, 49) (220390, 49)
(200136, 50) (210874, 50) (238130, 50) (220390, 50)


In [10]:
brfss_2021_mapped_cat.groupby(["Diabetes", "Age"]).size().reset_index().to_csv("ML4H 23 (Submission files)/Code/Group Sample Count/BRFSS - 2021 - Diabetes-age.csv", index = False)

# Utility Functions

## Performance Metrics

In [4]:
def performace_metrix(y_true, y_pred, y_pred_proba, group = "whole", disp = False):
  # Assuming you have true labels y_true and predicted labels y_pred
  # For class 0 and class 1

  #y_true, y_pred = y_test_2021, pred_test_2021_lr

  # Calculate precision, recall, and f1-score for each class
  precision_0 = precision_score(y_true, y_pred, pos_label=0)
  precision_1 = precision_score(y_true, y_pred, pos_label=1)

  recall_0 = recall_score(y_true, y_pred, pos_label=0)
  recall_1 = recall_score(y_true, y_pred, pos_label=1)

  f1_0 = f1_score(y_true, y_pred, pos_label=0)
  f1_1 = f1_score(y_true, y_pred, pos_label=1)

  # Calculate Area Under the Precision-Recall Curve for each class
  ap_0 = average_precision_score(y_true, y_pred_proba, pos_label=0)
  ap_1 = average_precision_score(y_true, y_pred_proba, pos_label=1)

  # Calculate overall accuracy
  accuracy = accuracy_score(y_true, y_pred)

  # Calculate balanced accuracy
  balanced_accuracy = balanced_accuracy_score(y_true, y_pred)

  # Calculate Area Under the ROC Curve
  roc_auc = roc_auc_score(y_true, y_pred_proba)

  # Calculate Matthews Correlation Coefficient (MCC)
  mcc = matthews_corrcoef(y_true, y_pred)

  if disp == True:
    # Print the calculated metrics
    print("Class 1 Recall:", recall_1)
    print("Class 1 Precision:", precision_1)
    print("Class 1 Area Under Precision-Recall Curve:", ap_1)
    print("Class 1 F1-Score:", f1_1)


    print("Class 0 Recall:", recall_0)
    print("Class 0 Precision:", precision_0)
    print("Class 0 Area Under Precision-Recall Curve:", ap_0)
    print("Class 0 F1-Score:", f1_0)

    print("Overall Accuracy:", accuracy)
    print("Balanced Accuracy:", balanced_accuracy)
    print("Area Under ROC Curve:", roc_auc)
    print("Matthews Correlation Coefficient:", mcc)


    # Print the calculated metrics
    print(recall_1)
    print(precision_1)
    print(ap_1)
    print(f1_1)

    print(recall_0)
    print(precision_0)
    print(ap_0)
    print(f1_0)

    print(accuracy)
    print(balanced_accuracy)
    print(roc_auc)
    print(mcc)


  performance = [group, recall_1, precision_1, ap_1, f1_1, recall_0, precision_0, ap_0, f1_0, accuracy, balanced_accuracy, roc_auc, mcc]
  return performance

def inset_performance_dict(performance_dict, performance):
    c = 0
    for k in performance_dict.keys():
        performance_dict[k].append(performance[c])
        c+=1
    return performance_dict

## Model Calibration

In [5]:
from sklearn.isotonic import IsotonicRegression as IR

def isotonic(preds, labels, test_preds):
    preds = np.array(preds)
    labels = np.array(labels)
    test_preds = np.array(test_preds)

    ir = IR(out_of_bounds='clip')
    ir.fit( preds, labels )

    p_calibrated_v = ir.transform( preds )
    p_calibrated_t = ir.transform( test_preds )   # or ir.fit( p_test ), that's the same thing

    return p_calibrated_v, p_calibrated_t

## Threshold Selection

In [6]:
import heapq
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score

def Select_Threshold(df):
    full_threshold_list = []
    for threshold in np.arange(0,1.05,0.05):
        #df.drop(columns = ['y_pred'])
        df['y_pred'] = df['calibrated_prediction_prob'].apply(lambda x: 1 if x >= threshold else 0)
        
        y_pred = df["y_pred"].values
        y_true = df["y_true"].values
        
        f1_C1 = f1_score(y_true, y_pred)
        balanced_accuracy = balanced_accuracy_score(y_true, y_pred)

        
        full_threshold_list.append([threshold, f1_C1, balanced_accuracy])
        
    df_varying_threshold = pd.DataFrame(full_threshold_list, columns = ['threshold', 'f1_score', 'balanced_accuracy'])
    
    # select three highest F1 score and the the highest balanced accuracy
    f1_scores = df_varying_threshold["f1_score"].values
    thresholds = df_varying_threshold["threshold"].values
    bal_acc_values = list(df_varying_threshold["balanced_accuracy"].values)
    
    #print(heapq.nlargest(3, f1_scores))
    list_index = heapq.nlargest(3, range(len(f1_scores)), key=f1_scores.__getitem__)
    opt_threshold = thresholds[bal_acc_values.index(max(bal_acc_values[list_index[0]], bal_acc_values[list_index[1]], bal_acc_values[list_index[2]]))]
    
    
    return opt_threshold, df_varying_threshold

def pred_label_using_threshold(y_pred_p, th = 0.195):
    y_pred_label = []
    for y in y_pred_p:
        if y <= th:
            y_pred_label.append(0)
        else:
            y_pred_label.append(1)
    return np.array(y_pred_label)

## Train and Test Model on BRFSS

In [10]:
def train_test_result(X_train, y_train, 
                         X_test, y_test, 
                         X_validation, y_validation,
                         df_test, 
                         save_dir = "Performance/",
                         model_name = "LR",
                         resampling = "No_sampling"):
    
    
    if resampling == "ENN":
      sampler = EditedNearestNeighbours(sampling_strategy='auto', n_neighbors=3)
    elif resampling == "NearMiss":
      sampler = NearMiss(sampling_strategy='auto', version=1)
    elif resampling == "SMOTE":
      sampler = SMOTE(sampling_strategy='auto', random_state=42)
    elif resampling == "RandomUnderSample":
      sampler = RandomUnderSampler(sampling_strategy='auto', random_state=42)
    elif resampling == "SMOTEENN":
      smote = SMOTE(sampling_strategy='auto', random_state=42)
      sampler = SMOTEENN(sampling_strategy='auto', smote=smote, random_state=42)
    elif resampling == "TomekLinks":
      sampler = TomekLinks(sampling_strategy='auto')
    elif resampling == "ADASYN":
      sampler = ADASYN(sampling_strategy='auto')
    elif resampling == "RandomOverSample":
      sampler = RandomOverSampler(sampling_strategy='minority')

    if resampling != "No_sampling":
      X_train, y_train = sampler.fit_resample(X_train, y_train)

    #print("Resampling", resampling)

    # train model
    if model_name == "LR":
      model = LogisticRegression(solver='liblinear')
    elif model_name == "RF":
      model = RandomForestClassifier()
    elif model_name == "SVM":
      model = SVC(probability=True, kernel='rbf', max_iter = 100)
    elif model_name == "KNN":
      model = KNeighborsClassifier(n_neighbors=5)
    elif model_name == "MLP":
      model = MLPClassifier()
    elif model_name == "NB":
      model = GaussianNB()
    elif model_name == "AdaBoost":
      model = AdaBoostClassifier(n_estimators=100, algorithm="SAMME", random_state=0)
    else:
      model = LogisticRegression(solver='liblinear')
    
    #print("model selected:", model_name)

    model.fit(X_train, y_train)

    # model predictions
    pred_prob_test_lr = model.predict_proba(X_test)[:, 1]
    pred_prob_validation_lr = model.predict_proba(X_validation)[:, 1]
        
    # model calibration
    ir = IR(out_of_bounds='clip')
    ir.fit(pred_prob_validation_lr, y_validation )
    pred_prob_validation_lr_calibrated = ir.transform( pred_prob_validation_lr )
    pred_prob_test_lr_calibrated = ir.transform( pred_prob_test_lr )

    # threshold selection
    val_preds = pd.DataFrame()
    val_preds['y_true'] = y_validation
    val_preds['calibrated_prediction_prob'] = pred_prob_validation_lr_calibrated
    threshold_val, _ = Select_Threshold(val_preds)
    #print("selected threshold:", threshold_val)
    
    # threshold based labeling
    pred_test_lr = pred_label_using_threshold(pred_prob_test_lr_calibrated, threshold_val)
    

    performance_dict = {
        "Group ": [],
        "Class 1 Recall": [],
        "Class 1 Precision": [],
        "Class 1 AU_PRC": [],
        "Class 1 F1-Score": [],
        "Class 0 Recall": [],
        "Class 0 Precision": [],
        "Class 0 AU_PRC": [],
        "Class 0 F1-Score": [],
        "Overall Accuracy": [],
        "Balanced Accuracy": [],
        "AU_ROC": [],
        "MCC": []
    }

    #print("============================ ", "Over all performance" , "============================ ")
    performance = performace_metrix(y_test, pred_test_lr, pred_prob_test_lr_calibrated, "Whole")
    performance_dict = inset_performance_dict(performance_dict, performance)

    # df_test contains "Race" column which is only required for group performance mapping
    test = df_test.copy()

    for i in range(1,3):
      if (i == 1):
        group = "Male"
        #print("============================ ", "Male" , "============================ ")
      else:
        group = "Female"
        #print("============================ ", "Female" , "============================ ")

      test_group = test[test.Sex == i]

      X_test_group = test_group.drop(['Diabetes', 'Race'],axis=1)
      y_test_group = test_group['Diabetes']
      y_test_group = y_test_group.to_numpy()

      #pred_test_group = lr.predict(X_test_group)
      pred_prob_test_group = model.predict_proba(X_test_group)[:, 1]
      
      #calibration
      pred_prob_test_group_calibrated = ir.transform( pred_prob_test_group )
    
      # thresholding
      pred_test_group = pred_label_using_threshold(pred_prob_test_group_calibrated, threshold_val)

      performance = performace_metrix(y_test_group, pred_test_group, pred_prob_test_group, group)
      performance_dict = inset_performance_dict(performance_dict, performance)


    race_list = ['White', 'Black','American Indian', 'Asian', 'Native Hawaiian',
                 'Other race', 'Multiracial', 'Hispanic']

    for i in race_list:
      test_group = test[test.Race == i]
      #print("============================ ", i , "============================ ")

      X_test_group = test_group.drop(['Diabetes', 'Race'],axis=1)
      y_test_group = test_group['Diabetes']
      y_test_group = y_test_group.to_numpy()

      #pred_test_group = lr.predict(X_test_group)
      pred_prob_test_group = model.predict_proba(X_test_group)[:, 1]
    
      #calibration
      pred_prob_test_group_calibrated = ir.transform( pred_prob_test_group )
    
      # thresholding
      pred_test_group = pred_label_using_threshold(pred_prob_test_group_calibrated, threshold_val)

      performance = performace_metrix(y_test_group, pred_test_group, pred_prob_test_group, i)
      performance_dict = inset_performance_dict(performance_dict, performance)


    age_group = {3: '30-34', 4: '35-39', 5: '40-44', 6: '45-49',
                 7: '50-54', 8: '55-59', 9: '60-64', 10: '65-69',
                 11: '70-74', 12: '75-79', 13: '80-99'}

    for i in age_group.keys():
      test_group = test[test.Age == i]
      #print("============================ ", age_group[i] , "============================ ")

      X_test_group = test_group.drop(['Diabetes', 'Race'],axis=1)
      y_test_group = test_group['Diabetes']
      y_test_group = y_test_group.to_numpy()

      #pred_test_group = lr.predict(X_test_group)
      pred_prob_test_group = model.predict_proba(X_test_group)[:, 1]
    
      #calibration
      pred_prob_test_group_calibrated = ir.transform( pred_prob_test_group )
    
      # thresholding
      pred_test_group = pred_label_using_threshold(pred_prob_test_group_calibrated, threshold_val)

      performace = performace_metrix(y_test_group, pred_test_group, pred_prob_test_group, age_group[i])
      performance_dict = inset_performance_dict(performance_dict, performace)

    all_performances = pd.DataFrame(performance_dict)
    all_performances.to_csv(save_dir + model_name + "_" + resampling   +  ".csv", index=False)
    
    return all_performances

    

# Create Data split for entire experiment

## 2021

In [8]:
random_state = 1
df_train_2021, df_test_2021 = train_test_split(brfss_2021_one_hot, test_size=0.4, random_state=random_state)
df_test_2021, df_validation_2021 = train_test_split(df_test_2021, test_size=0.5, random_state=random_state)
print(brfss_2021_one_hot.shape, df_train_2021.shape, df_test_2021.shape, df_validation_2021.shape)

X_train_2021 = df_train_2021.drop(['Diabetes', 'Race'],axis=1)
y_train_2021 = df_train_2021['Diabetes']

X_test_2021 = df_test_2021.drop(['Diabetes', 'Race'],axis=1)
y_test_2021 = df_test_2021['Diabetes']
y_test_2021 = y_test_2021.to_numpy()

X_validation_2021 = df_validation_2021.drop(['Diabetes', 'Race'],axis=1)
y_validation_2021 = df_validation_2021['Diabetes']
y_validation_2021 = y_validation_2021.to_numpy()

print(X_train_2021.shape, X_test_2021.shape, X_validation_2021.shape)

(200136, 50) (120081, 50) (40027, 50) (40028, 50)
(120081, 48) (40027, 48) (40028, 48)


## 2015

In [9]:
random_state = 1
df_train_2015, df_test_2015 = train_test_split(brfss_2015_one_hot, test_size=0.4, random_state=random_state)
df_test_2015, df_validation_2015 = train_test_split(df_test_2015, test_size=0.5, random_state=random_state)
print(brfss_2015_one_hot.shape, df_train_2015.shape, df_test_2015.shape, df_validation_2015.shape)

X_train_2015 = df_train_2015.drop(['Diabetes', 'Race'],axis=1)
y_train_2015 = df_train_2015['Diabetes']

X_test_2015 = df_test_2015.drop(['Diabetes', 'Race'],axis=1)
y_test_2015 = df_test_2015['Diabetes']
y_test_2015 = y_test_2015.to_numpy()

X_validation_2015 = df_validation_2015.drop(['Diabetes', 'Race'],axis=1)
y_validation_2015 = df_validation_2015['Diabetes']
y_validation_2015 = y_validation_2015.to_numpy()

print(X_train_2015.shape, X_test_2015.shape, X_validation_2015.shape)

(220390, 50) (132234, 50) (44078, 50) (44078, 50)
(132234, 48) (44078, 48) (44078, 48)


# Age Based Subsets

## 2021

In [ ]:
import os


age_group_map = {
    3:	'30-34',
    4:	'35-39',
    5:	'40-44',
    6:	'45-49',
    7:	'50-54',
    8:	'55-59',
    9:	'60-64',
    10:	'65-69',
    11:	'70-74',
    12:	'75-79',
    13:	'80-99'
}

for age_group in age_group_map.keys():
    print("===========================", age_group_map[age_group], "===========================")

    Train_age_3_df = df_train_2021[df_train_2021['Age'] == age_group]
    Validation_age_3_df = df_validation_2021[df_validation_2021['Age'] == age_group]
    Test_age_3_df = df_test_2021[df_test_2021['Age'] == age_group]

    print(Train_age_3_df.shape, Validation_age_3_df.shape, Test_age_3_df.shape)

    X_train = Train_age_3_df.drop(['Diabetes', 'Race'],axis=1)
    y_train = Train_age_3_df['Diabetes']

    X_test = Test_age_3_df.drop(['Diabetes', 'Race'],axis=1)
    y_test = Test_age_3_df['Diabetes']

    X_val = Validation_age_3_df.drop(['Diabetes', 'Race'],axis=1)
    y_val = Validation_age_3_df['Diabetes']


    print(X_train.shape, X_train.shape, X_test.shape)

    model_names = ["LR", "RF", "SVM", "KNN", "MLP", "NB", "AdaBoost"]
    resamplings = ["ENN", "NearMiss", "SMOTE", "RandomUnderSample", "SMOTEENN", "TomekLinks", "ADASYN", "RandomOverSample", "No_sampling"]

    directory = "ML4H 23 (Submission files)/Code/Subgroup_separate_training/BRFSS_2021/Age/" + age_group_map[age_group] + "/"
    if not os.path.exists(directory):
        os.makedirs(directory)

    for model_name in model_names:
        for resampling in resamplings:

            train_test_result(X_train, y_train, 
                                X_test, y_test,  
                                X_val, y_val,
                                df_test_2021,
                                save_dir = directory,
                                model_name = model_name,
                                resampling = resampling)

In [12]:
import os


age_group_map = {
    '30-44': [3, 4, 5],
    '45-59': [6, 7, 8],
    '60-74': [9, 10, 11],
    '75-99': [12, 13]
}


for age_group in age_group_map.keys():
    print("===========================", age_group, "===========================")

    Train_age_3_df = df_train_2021[df_train_2021['Age'].isin(age_group_map[age_group])]
    Validation_age_3_df = df_validation_2021[df_validation_2021['Age'].isin(age_group_map[age_group])]
    Test_age_3_df = df_test_2021[df_test_2021['Age'].isin(age_group_map[age_group])]

    print(Train_age_3_df.shape, Validation_age_3_df.shape, Test_age_3_df.shape)

    X_train = Train_age_3_df.drop(['Diabetes', 'Race'],axis=1)
    y_train = Train_age_3_df['Diabetes']

    X_test = Test_age_3_df.drop(['Diabetes', 'Race'],axis=1)
    y_test = Test_age_3_df['Diabetes']

    X_val = Validation_age_3_df.drop(['Diabetes', 'Race'],axis=1)
    y_val = Validation_age_3_df['Diabetes']


    print(X_train.shape, X_train.shape, X_test.shape)

    model_names = ["LR", "RF", "SVM", "KNN", "MLP", "NB", "AdaBoost"]
    resamplings = ["ENN", "NearMiss", "SMOTE", "RandomUnderSample", "SMOTEENN", "TomekLinks", "ADASYN", "RandomOverSample", "No_sampling"]

    directory = "ML4H 23 (Submission files)/Code/Subgroup_separate_training/BRFSS_2021/Age/span_15/" + age_group + "/"

    if not os.path.exists(directory):
        os.makedirs(directory)

    for model_name in model_names:
        for resampling in resamplings:

            train_test_result(X_train, y_train, 
                                X_test, y_test,  
                                X_val, y_val,
                                df_test_2021,
                                save_dir = directory,
                                model_name = model_name,
                                resampling = resampling)

=========================== 30-44 ===========================
(24399, 50) (8214, 50) (8177, 50)
(24399, 48) (24399, 48) (8177, 48)
=========================== 45-59 ===========================
(34452, 50) (11339, 50) (11400, 50)
(34452, 48) (34452, 48) (11400, 48)
=========================== 60-74 ===========================
(43925, 50) (14745, 50) (14649, 50)
(43925, 48) (43925, 48) (14649, 48)
=========================== 75-99 ===========================
(17305, 50) (5730, 50) (5801, 50)
(17305, 48) (17305, 48) (5801, 48)


## 2015

In [ ]:
import os


age_group_map = {
    3:	'30-34',
    4:	'35-39',
    5:	'40-44',
    6:	'45-49',
    7:	'50-54',
    8:	'55-59',
    9:	'60-64',
    10:	'65-69',
    11:	'70-74',
    12:	'75-79',
    13:	'80-99'
}

for age_group in age_group_map.keys():
    print("===========================", age_group_map[age_group], "===========================")

    Train_age_3_df = df_train_2015[df_train_2015['Age'] == age_group]
    Validation_age_3_df = df_validation_2015[df_validation_2015['Age'] == age_group]
    Test_age_3_df = df_test_2015[df_test_2015['Age'] == age_group]

    print(Train_age_3_df.shape, Validation_age_3_df.shape, Test_age_3_df.shape)

    X_train = Train_age_3_df.drop(['Diabetes', 'Race'],axis=1)
    y_train = Train_age_3_df['Diabetes']

    X_test = Test_age_3_df.drop(['Diabetes', 'Race'],axis=1)
    y_test = Test_age_3_df['Diabetes']

    X_val = Validation_age_3_df.drop(['Diabetes', 'Race'],axis=1)
    y_val = Validation_age_3_df['Diabetes']


    print(X_train.shape, X_train.shape, X_test.shape)

    model_names = ["LR", "RF", "SVM", "KNN", "MLP", "NB", "AdaBoost"]
    resamplings = ["ENN", "NearMiss", "SMOTE", "RandomUnderSample", "SMOTEENN", "TomekLinks", "ADASYN", "RandomOverSample", "No_sampling"]

    directory = "ML4H 23 (Submission files)/Code/Subgroup_separate_training/BRFSS_2015/Age/" + age_group_map[age_group] + "/"
    if not os.path.exists(directory):
        os.makedirs(directory)

    for model_name in model_names:
        for resampling in resamplings:

            train_test_result(X_train, y_train, 
                                X_test, y_test,  
                                X_val, y_val,
                                df_test_2015,
                                save_dir = directory,
                                model_name = model_name,
                                resampling = resampling)



In [13]:
import os


age_group_map = {
    '30-44': [3, 4, 5],
    '45-59': [6, 7, 8],
    '60-74': [9, 10, 11],
    '75-99': [12, 13]
}


for age_group in age_group_map.keys():
    print("===========================", age_group, "===========================")

    Train_age_3_df = df_train_2015[df_train_2015['Age'].isin(age_group_map[age_group])]
    Validation_age_3_df = df_validation_2015[df_validation_2015['Age'].isin(age_group_map[age_group])]
    Test_age_3_df = df_test_2015[df_test_2015['Age'].isin(age_group_map[age_group])]

    print(Train_age_3_df.shape, Validation_age_3_df.shape, Test_age_3_df.shape)

    X_train = Train_age_3_df.drop(['Diabetes', 'Race'],axis=1)
    y_train = Train_age_3_df['Diabetes']

    X_test = Test_age_3_df.drop(['Diabetes', 'Race'],axis=1)
    y_test = Test_age_3_df['Diabetes']

    X_val = Validation_age_3_df.drop(['Diabetes', 'Race'],axis=1)
    y_val = Validation_age_3_df['Diabetes']


    print(X_train.shape, X_train.shape, X_test.shape)

    model_names = ["LR", "RF", "SVM", "KNN", "MLP", "NB", "AdaBoost"]
    resamplings = ["ENN", "NearMiss", "SMOTE", "RandomUnderSample", "SMOTEENN", "TomekLinks", "ADASYN", "RandomOverSample", "No_sampling"]

    directory = "ML4H 23 (Submission files)/Code/Subgroup_separate_training/BRFSS_2015/Age/span_15/" + age_group + "/"

    if not os.path.exists(directory):
        os.makedirs(directory)

    for model_name in model_names:
        for resampling in resamplings:

            train_test_result(X_train, y_train, 
                                X_test, y_test,  
                                X_val, y_val,
                                df_test_2015,
                                save_dir = directory,
                                model_name = model_name,
                                resampling = resampling)

=========================== 30-44 ===========================
(22096, 50) (7572, 50) (7479, 50)
(22096, 48) (22096, 48) (7479, 48)
=========================== 45-59 ===========================
(42454, 50) (13994, 50) (14122, 50)
(42454, 48) (42454, 48) (14122, 48)
=========================== 60-74 ===========================
(49465, 50) (16379, 50) (16434, 50)
(49465, 48) (49465, 48) (16434, 48)
=========================== 75-99 ===========================
(18219, 50) (6133, 50) (6043, 50)
(18219, 48) (18219, 48) (6043, 48)
